In [1]:
# Importação das bibliotecas necessárias
import pandas as pd

from bs4 import BeautifulSoup
import requests

import pdfplumber

In [5]:
fonte = 'mapbiomas'
dataset_name = 'uso_terra'
raiz = '/home/eduardo/repositorios/IS_Agro/databases'

### Bronze

In [3]:
# Requisição para a URL
response = requests.get('https://brasil.mapbiomas.org/estatisticas/')

# Criação do objeto BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Listagem de todos os links na página
links = soup.find_all('a')

In [ ]:
# Filtro para apenas os links de download
download_links = [link.get('href') for link in links if 'download' in link.get('href') and 'COBERTURA' in link.text and 'MUNICÍPIO' in link.text]

# Seleção do link de download (caso o filtro pegue outros links que atendam o critério) com aquisição apenas do nome do arquivo
cobertura_xlsx = download_links[0].split("/")[-1]

# Salvamento do arquivo em disco com o nome original, assim mantendo a coleção
with open(f'{raiz}/bronze_{cobertura_xlsx}', 'wb') as arquivo:
    arquivo.write(requests.get(download_links[0]).content)

### Prata

In [4]:
# Obtenção de lista com anos de interesse que constam como colunas da tabela
anos = (pd.read_excel(f'{raiz}/bronze_{cobertura_xlsx}',
                      sheet_name = 1,
                      usecols=lambda x: isinstance(x, int))
        .columns
        .drop(range(1985, 1990))
        .tolist()
        )

In [ ]:
df = (
    pd.read_excel(f'{raiz}/bronze_{cobertura_xlsx}',
                      sheet_name = 1)
    .drop(range(1985, 1990), axis = 'columns')
    .rename(columns=lambda col: 'class' if col == 'class_id' else col)
    .melt(
        id_vars=['geocode', 'class'],
        value_vars=anos,
        var_name='data',
        value_name='area'
        )
    .dropna(subset='area')
    #.query('area != 0.0')
    .assign(data = lambda df: pd.to_datetime(df.data, format='%Y'), # Conversão de número inteiro para formato de data seguindo a ISO 8601
                                ) 
    .convert_dtypes()
    .rename(columns=
            {
                'geocode':'geocodigo',
                'class':'classe'
             })
    )
(
    df.set_index(['geocodigo', 'data', 'classe'])
    .to_parquet(f'{raiz}/prata_{fonte}_{dataset_name}_municipio.parquet')
)

df_uf = (df
         .assign(geocodigo_uf = lambda df: df['geocodigo']// 100000)
         .groupby(['geocodigo_uf', 'data', 'classe'])
         .agg({'area':sum})
         .rename_axis(index={'geocodigo_uf': 'geocodigo'})
         .convert_dtypes()
         )
df_uf.to_parquet(f'{raiz}/prata_{fonte}_{dataset_name}_uf.parquet')

### Legenda

##### Bronze

In [2]:
# Requisição para a URL
response = requests.get('https://brasil.mapbiomas.org/codigos-de-legenda/')

# Criação do objeto BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Listagem de todos os links na página
links = soup.find_all('a')

In [3]:
# Encontrar o primeiro link para PDF
pdf_link = [link.get('href') for link in links if 'pdf' in link.get('href').lower()][0]

# Extração do nome do arquivo do PDF
pdf_filename = pdf_link.split("/")[-1]

# Salvamento do arquivo em disco
# with open(f'{raiz}/bronze_{pdf_filename}', 'wb') as arquivo:
#     arquivo.write(requests.get(pdf_link).content)

##### Prata

In [6]:
# Abrir e copiar a tabela do PDF
with pdfplumber.open(f'{raiz}/bronze_{pdf_filename}') as pdf:
    tabela = pdf.pages[0].extract_table() # Leitura da primeira página e extração da tabela

In [11]:
# Criação da tabela de códigos do MapBiomas
df_codigo = (
    pd.DataFrame(tabela)
    .filter([3, 1])
    .rename(columns={3: 'codigo', 1: 'classe_cod'})
    .drop(index=[0,1])
    .replace('27', '0')
    .assign(
        rotulo=lambda x: x.classe_cod.str.replace(r'[^A-Za-zÀ-ÿ\s]', '', regex=True).str.strip(),
        codigo=lambda n: n.codigo.astype(int)
    )
    .convert_dtypes()
    .reset_index(drop=True)
)

df_codigo.to_parquet(f'{raiz}/prata_{fonte}_codigo_{dataset_name}.parquet', index=False)

In [12]:
df_codigo.to_csv(f'{raiz}/prata_{fonte}_codigo_{dataset_name}.csv', index=False)